In [1]:
import numpy as np
import pandas as pd
import multiprocessing as mp


In [7]:
# load labels
labels = pd.read_csv('../data/Lauren/Labels.csv')
labels.head() # to display the first 5 lines of loaded data

,Class,Subclass,cluster
0,GABAergic,Vip,Vip Arhgap36 Hmcn1
1,GABAergic,Lamp5,Lamp5 Lsp1
2,GABAergic,Lamp5,Lamp5 Lsp1
3,GABAergic,Vip,Vip Crispld2 Htr2c
4,GABAergic,Lamp5,Lamp5 Plch2 Dock5


In [5]:
#%%time # te traag
# load data https://towardsdatascience.com/%EF%B8%8F-load-the-same-csv-file-10x-times-faster-and-with-10x-less-memory-%EF%B8%8F-e93b485086c7
#df = pd.read_csv('../data/Lauren/500_PBMC_3p_LT_Chromium_X_metrics_summary.csv', chunksize = 1000)
#total_length = 0
#result = 0
#for chunk in df:
#    if type(result) == int:
#        result = f.get()
#    else:
#        result = pd.concat([result, f.get()], sort=False)
#len(result)
#data.head() # to display the first 5 lines of loaded data

In [2]:
%%time

# loading in the file in different chunks

LARGE_FILE = "../data/Lauren/500_PBMC_3p_LT_Chromium_X_metrics_summary.csv"
CHUNKSIZE = 1000 # processing 100,000 rows at a time: this is a documentation comment from the source, but i dont think its correct

def process_frame(df):
    # here we can do extra preprocessing steps in parallel
    return df

reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE)

pool = mp.Pool(6) # use 4 processes
funclist = []
for df in reader:    
    # process each data frame
    f = pool.apply_async(process_frame,[df])
    funclist.append(f)
   
        

CPU times: user 4min 40s, sys: 49.7 s, total: 5min 29s
Wall time: 4min 38s


In [3]:
%%time

# combining the different chunks into 1 big dataframe

df_list = []
for f in funclist:
    df_list.append(f.get())
    
unfiltered_df = pd.concat(df_list, ignore_index=True)

print(len(unfiltered_df))

12832
CPU times: user 1 s, sys: 809 ms, total: 1.81 s
Wall time: 2.24 s


In [4]:
unfiltered_df

,Unnamed: 0,0610005C13Rik,0610006L08Rik,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010B08Rik,0610010F05Rik,...,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,n-R5s136
0,F1S4_160108_001_A01,0,0,79,145,1,46,123,0,0,...,0,0,0,78,0,36,1,0,190,0
1,F1S4_160108_001_B01,0,0,123,178,26,23,2,0,337,...,0,0,0,69,0,78,29,0,179,0
2,F1S4_160108_001_C01,0,0,89,69,0,13,29,0,112,...,0,0,2,273,38,110,81,0,178,0
3,F1S4_160108_001_D01,0,0,115,0,0,0,50,0,35,...,0,0,0,151,165,67,30,0,110,0
4,F1S4_160108_001_E01,0,0,402,208,0,42,0,0,0,...,20,0,1,99,50,83,69,0,335,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12827,FYS4_171004_103_H01,0,0,45,82,0,10,0,0,23,...,0,0,0,115,0,25,0,0,143,0
12828,FYS4_171004_104_A01,0,0,88,79,0,0,1,0,95,...,0,17,0,92,0,1,26,0,94,0
12829,FYS4_171004_104_B01,0,0,81,105,0,0,78,0,104,...,0,10,0,77,7,62,0,0,147,0
12830,FYS4_171004_104_C01,0,0,187,52,0,8,4,0,8,...,0,24,0,54,67,0,70,0,133,0


In [14]:
## Filter labels
filter_on = "Class"
amount_higher_than = 10


classes_count = labels.groupby(filter_on).count()
print(classes_count)
print()
# the count is in last column: classes_count[classes_count.columns[-1]
classes_to_keep = list(classes_count[classes_count[classes_count.columns[-1]] >= amount_higher_than].index)
print(classes_to_keep)
print()
indices = labels[filter_on].isin(classes_to_keep)
print(indices.all())
labels_filtered = labels[indices]
print(labels.shape)
print(labels_filtered.shape)



               Subclass  cluster
Class                           
Endothelial           6        6
GABAergic          5643     5643
Glutamatergic      7155     7155
Non-Neuronal         28       28

['GABAergic', 'Glutamatergic', 'Non-Neuronal']

False
(12832, 3)
(12826, 3)


In [16]:
## Filter data
df = unfiltered_df[indices]
print(unfiltered_df.shape)
print(df.shape)
df # the indexes are not correct anymore, because they didn't change with the deletion of certain rows

(12832, 42626)
(12826, 42626)


,Unnamed: 0,0610005C13Rik,0610006L08Rik,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010B08Rik,0610010F05Rik,...,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,n-R5s136
0,F1S4_160108_001_A01,0,0,79,145,1,46,123,0,0,...,0,0,0,78,0,36,1,0,190,0
1,F1S4_160108_001_B01,0,0,123,178,26,23,2,0,337,...,0,0,0,69,0,78,29,0,179,0
2,F1S4_160108_001_C01,0,0,89,69,0,13,29,0,112,...,0,0,2,273,38,110,81,0,178,0
3,F1S4_160108_001_D01,0,0,115,0,0,0,50,0,35,...,0,0,0,151,165,67,30,0,110,0
4,F1S4_160108_001_E01,0,0,402,208,0,42,0,0,0,...,20,0,1,99,50,83,69,0,335,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12827,FYS4_171004_103_H01,0,0,45,82,0,10,0,0,23,...,0,0,0,115,0,25,0,0,143,0
12828,FYS4_171004_104_A01,0,0,88,79,0,0,1,0,95,...,0,17,0,92,0,1,26,0,94,0
12829,FYS4_171004_104_B01,0,0,81,105,0,0,78,0,104,...,0,10,0,77,7,62,0,0,147,0
12830,FYS4_171004_104_C01,0,0,187,52,0,8,4,0,8,...,0,24,0,54,67,0,70,0,133,0
